In [13]:
import os
import datetime
from glob import iglob,glob

import xarray as xr
import numpy as np
import pandas as pd

from params import *

In [14]:
def raise_incorrect_type(which):
    """
    Raise value error if incorrect forecast type

    Args:
        which (str): kind of forecast (atm, ocean ,wave)
    """
    if which not in ['atm','ocean','wave']:
        raise ValueError('Which parameter must indicate only some\
                          of these ["ocean","wave", "atm"]') 
    return

def first_forecast_path(which):
    """
    Return the first aviable forecast of each type

    Args:
        which (str): kind of forecast (atm, ocean ,wave)

    Returns:
        str: path to netcdf file
    """
    path = glob(eval(f'{which}_globalparams')['forecast_dir']+'/*.nc')
    path = sorted(path)[0]
    return path

def first_forecast_date(which):
    """
    Return the date of the first aviable forecast of each type

    Args:
        which (str): kind of forecast (atm, ocean ,wave)

    Returns:
        str: first forecast date
    """
    path = first_forecast_path(which)
    fdir     = eval(f'{which}_globalparams')['forecast_dir']
    fsuffix  = eval(f'{which}_globalparams')['forecast_suffix']
    date = path.replace(fdir,'')
    date = date.replace(fsuffix,'')
    date = date.replace('/','')
    date = date.replace('.nc','')
    date = pd.to_datetime(date).strftime('%F')
    return date
    

def last_forecast_path(which):
    """
    Return last aviable forecast of each type
    Args:
        which (str): kind of forecast (atm, ocean ,wave)

    Returns:
        str: path to netcdf file
    """
    path = glob(eval(f'{which}_globalparams')['forecast_dir']+'/*.nc')
    path = sorted(path)[-1]

    return path


def forecast_path(date, which):
    """
    Return forecast path for a type and specific date.
    
    Args:
        which (str): type of forecast (atm, ocean ,wave)
        date  (str): %Y-%m-%d

    Returns:
        str: path to netcdf file
    """
    # Some checks
    raise_incorrect_type(which)
    
    date      = pd.to_datetime(date)
    firstdate = pd.to_datetime(first_forecast_date(which)) 
    # If date goes before any aviable forecast raise error
    if date<firstdate:
        raise RuntimeError(f'Forecast are only aviable for dates after {firstdate}')
    # If date is from today or the future return last valid forecast
    if pd.to_datetime(date).date() >= datetime.datetime.now().date():
        path = last_forecast_path(which)
    else:
        fdir     = eval(f'{which}_globalparams')['forecast_dir']
        fsuffix  = eval(f'{which}_globalparams')['forecast_suffix']
        fdformat = eval(f'{which}_globalparams')['forecast_dateformat']
        path     = '{}/{}'.format(fdir,fsuffix)
        path     = path+'{}.nc'.format(date.strftime(fdformat))
    
    # Check if constructed path exist else return nearest previous forecast
    # and raise warning file
    if os.path.isfile(path):
        # All good
        pass
    else:
        ndate = date-pd.Timedelta(days=1)
        print(f'{path} doesnt exists!',
              'Trying with {} forecast'.format(ndate.strftime('%F')))
        return forecast_path(ndate, which)
    
    return path

In [15]:
def load_wrf(path, **kwargs):
    """ 
    Load a wrf model output and adjus
    for the package to work.
    Args:
        path (str): path to netcdf file
        **kwargs passed to xarray.open_dataset

    Returns:
        data (xDataset): loaded xarray
    """
    data  = xr.open_dataset(path, **kwargs)
    cdict = {value:key for key, value in atm_variables.items()}
    data  = data.rename(cdict)[atm_variables.keys()]
    
    inittime = data.attrs['SIMULATION_START_DATE'].replace('_',' ')
    data.coords['inittime'] = pd.to_datetime(inittime) 
    return data

In [16]:
def load_mercator_ocean(path, **kwargs):
    """ 
    load a mercator model output and adjust
    for the package to work.
    Args:
        path (str): path to netcdf file
        **kwargs passed to xarray.open_dataset

    Returns:
        data (xDataset): loaded xarray
    """
    data = xr.open_dataset(path, engine='netcdf4', **kwargs)
    data = data.squeeze()
    cdict = {value:key for key, value in ocean_variables.items()}
    data  = data.rename(cdict)[ocean_variables.keys()]
    p = path.split("/")[-1].split(".")[0]
    data.coords['inittime'] = pd.to_datetime(p,format="%Y-%m-%d")
    return data

def load_mercator_waves(path, **kwargs):
    """ 
    load a mercator model output and adjust
    for the package to work.
    Args:
        path (str): path to netcdf file
        **kwargs passed to xarray.open_dataset

    Returns:
        data (xDataset): loaded xarray
    """
    data = xr.open_dataset(path, engine='netcdf4', **kwargs)
    data = data.squeeze()
    cdict = {value:key for key, value in wave_variables.items()}
    data  = data.rename(cdict)[wave_variables.keys()]
    p = path.split("/")[-1].split(".")[0]
    data.coords['inittime'] = pd.to_datetime(p,format="%Y-%m-%d")
    return data

In [17]:
path = forecast_path('2023-08-10','ocean')
data  = load_mercator(path)

In [18]:
data

<xarray.Dataset>
Dimensions:   (time: 240, lat: 301, lon: 121)
Coordinates:
  * time      (time) datetime64[ns] 2023-08-10T00:30:00 ... 2023-08-19T23:30:00
  * lat       (lat) float32 -40.0 -39.92 -39.83 -39.75 ... -15.17 -15.08 -15.0
  * lon       (lon) float32 -80.0 -79.92 -79.83 -79.75 ... -70.17 -70.08 -70.0
    depth     float32 0.494
    inittime  datetime64[ns] 2023-08-10
Data variables:
    temp      (time, lat, lon) float32 ...
    salt      (time, lat, lon) float32 ...
    u         (time, lat, lon) float32 ...
    v         (time, lat, lon) float32 ...
    ssh       (time, lat, lon) float32 ...
Attributes: (12/15)
    producer:                  CMEMS - Global Monitoring and Forecasting Centre
    title:                     hourly mean fields from Global Ocean Physics A...
    area:                      Global
    Conventions:               CF-1.8
    credit:                    E.U. Copernicus Marine Service Information (CM...
    contact:                   https://marine.copernicus.eu/contact
    ...                        ...
    institution:               Mercator Ocean International
    _CoordSysBuilder:          ucar.nc2.dataset.conv.CF1Convention
    comment:                   
    history:                   Thu Aug 10 09:46:12 2023: ncpdq -L 5 -7 2023-0...
    NCO:                       4.7.2
    nco_openmp_thread_number:  1